In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
from textblob import TextBlob
import requests

# Функция для анализа текста
def analyze_text():
    input_text = text_area.get("1.0", tk.END).strip()  # Получаем текст из поля
    if not input_text:
        messagebox.showerror("Ошибка", "Пожалуйста, введите текст для анализа.")
        return

    blob = TextBlob(input_text)

    # Анализ предложений
    sentence_analysis = f"Количество предложений: {len(blob.sentences)}\n"
    
    # Подсчет слов
    words_analysis = f"Количество слов: {len(blob.words)}\n"
    
    # Поиск имен (предположим, что это будут имена собственные с заглавной буквы)
    names = [word for word in blob.words if word.istitle()]
    names_analysis = f"Найденные имена: {', '.join(names) if names else 'Нет'}\n"
    
    # Поиск слов, начинающихся с определенной буквы (например, 'W')
    starts_with_letter = [word for word in blob.words if word.lower().startswith(start_letter.get())]
    starts_with_analysis = f"Слова, начинающиеся на '{start_letter.get()}': {', '.join(starts_with_letter) if starts_with_letter else 'Нет'}\n"

    # Вывод результатов
    result_text.set(sentence_analysis + words_analysis + names_analysis + starts_with_analysis)

# Функция для загрузки текста с сайта Gutenberg
def load_text_from_gutenberg():
    url = 'https://www.gutenberg.org/files/514/514-0.txt'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки при загрузке
        text = response.text
        start_idx = text.find("*** START OF THIS PROJECT GUTENBERG EBOOK LITTLE WOMEN ***") + len("*** START OF THIS PROJECT GUTENBERG EBOOK LITTLE WOMEN ***")
        end_idx = text.find("*** END OF THIS PROJECT GUTENBERG EBOOK LITTLE WOMEN ***")
        book_text = text[start_idx:end_idx].strip()  # Извлекаем текст между этими отметками

        text_area.delete("1.0", tk.END)  # Очищаем текстовое поле
        text_area.insert(tk.END, book_text)  # Вставляем текст из книги
    except requests.RequestException as e:
        messagebox.showerror("Ошибка", f"Не удалось загрузить текст: {e}")

# Создаем окно Tkinter
root = tk.Tk()
root.title("Анализ текста: Маленькие женщины")

# Текстовое поле для ввода
tk.Label(root, text="Введите текст для анализа:").pack()
text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=50, height=10)
text_area.pack(padx=10, pady=10)

# Кнопка для загрузки текста с сайта Gutenberg
load_button = tk.Button(root, text="Загрузить текст с сайта Gutenberg", command=load_text_from_gutenberg)
load_button.pack(pady=5)

# Поле для ввода первой буквы
tk.Label(root, text="Введите букву для поиска слов, начинающихся с неё:").pack()
start_letter = tk.StringVar(value='w')
tk.Entry(root, textvariable=start_letter).pack(pady=5)

# Кнопка для запуска анализа
analyze_button = tk.Button(root, text="Анализировать", command=analyze_text)
analyze_button.pack(pady=10)

# Поле для вывода результатов анализа
tk.Label(root, text="Результаты анализа:").pack()
result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, justify=tk.LEFT)
result_label.pack(padx=10, pady=10)

# Запуск основного цикла Tkinter
root.mainloop()